In [2]:
import boto3
import sagemaker
import pickle
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

In [3]:
boto_session = boto3.Session(profile_name='ml_predict',region_name='ap-northeast-1')
sess = sagemaker.Session(boto_session=boto_session)

In [3]:
endpoint_name = 'tf-complex-cnn-2021-07-30-16-20-57-143'

predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_name, 
                                          sagemaker_session=sess, se)

In [8]:
agg = pd.read_csv('agg_revised.csv')

In [10]:
# with open('onehot_encoder.pkl', 'rb') as f:
#     ohe = pickle.load(f)

ohe = OneHotEncoder(sparse=False, drop='first')
ohe.fit(agg[['age', 'gender', 'season']])

OneHotEncoder(categorical_features=None, categories=None, drop='first',
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=False)

In [11]:
img_path = 'unique_images/50.png'

In [13]:
img_input = np.array([np.array(Image.open(img_path).resize((128,128)))])
brf_input = ohe.transform([['18-34', 'male', 1]])
brf_input = tf.expand_dims(np.append(brf_input, 30), axis=0)

In [39]:
predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

In [20]:
input_data = {
    'inputs': {
        'input_1': img_input.tolist(), 
        'input_2': brf_input.numpy().tolist()
    }
}

In [40]:
pred = predictor.predict(input_data)

In [41]:
pred

{'outputs': [[2.65237093]]}